### Run upon export from spreadsheet

In [ ]:
import glob
import os
import random
import time

from astropy.io import fits
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [ ]:
final = pd.DataFrame()

labels = pd.read_csv('/mnt/tess/labels/labels-triage-v13.csv', header=0, low_memory=False)
labels = labels[['TIC ID', 'Final', 'av', 'md', 'ch', 'as', 'mk', 'et', 'dm', 'td']]
raters = ['av', 'md', 'ch', 'as', 'mk', 'et', 'dm', 'td']

tr6 = pd.read_csv('/mnt/tess/astronet/tces-v6-train.csv', header=0, low_memory=False)
tr7 = pd.read_csv('/mnt/tess/astronet/tces-v7-train.csv', header=0, low_memory=False)
vl6 = pd.read_csv('/mnt/tess/astronet/tces-v6-val.csv', header=0, low_memory=False)
vl7 = pd.read_csv('/mnt/tess/astronet/tces-v7-val.csv', header=0, low_memory=False)
ts6 = pd.read_csv('/mnt/tess/astronet/tces-v6-test.csv', header=0, low_memory=False)
ts7 = pd.read_csv('/mnt/tess/astronet/tces-v7-test.csv', header=0, low_memory=False)

origtce = pd.read_csv('/mnt/tess/labels/tce_bls_instar.csv', header=0, low_memory=False)
origtce['disp_E'] = 0
origtce['disp_S'] = 0
origtce['disp_B'] = 0
origtce['disp_J'] = 0
origtce['disp_N'] = 0
origtce['Duration'] /= 24.0
origtce2 = pd.read_csv('/mnt/tess/labels/tce_bls_instar+old.csv', header=0, low_memory=False)
origtce2['disp_E'] = 0
origtce2['disp_S'] = 0
origtce2['disp_B'] = 0
origtce2['disp_J'] = 0
origtce2['disp_N'] = 0
origtce2['Duration'] /= 24.0

newtce = pd.read_csv('/mnt/tess/labels/tces-triage-v12.csv', header=0, low_memory=False)

mast = pd.read_csv('/mnt/tess/labels/ext_mast_data.csv', header=0, low_memory=False)


print('...', end='')
minidx = 0

for idx, rec in labels.iterrows():
  ticid = rec['TIC ID']
  if idx < minidx:
    continue

  rec = rec.copy()
  tc = tr6[tr6['TIC ID'] == ticid]
  tc7 = tr7[tr7['TIC ID'] == ticid]
  split = 'train'
  
  if not len(tc):
    tc = vl6[vl6['TIC ID'] == ticid]
    tc7 = vl7[vl7['TIC ID'] == ticid]
    split = 'val'
  if not len(tc):
    tc = ts6[ts6['TIC ID'] == ticid]
    tc7 = ts7[ts7['TIC ID'] == ticid]
    split = 'test'

  obj_type = mast[mast['tic_id'] == ticid]['objType']
  
  isy3 = idx > 26380

  if isy3:
    tc = newtce[(newtce['tic_id'] == ticid)
              & (newtce['filename'].str.startswith('mk_'))]
    assert len(tc) == 1, ticid
    split = tc['Split'].values[0]
    assert split in ('train', 'test', 'val')
  else:
    if len(tc):
      assert len(tc) == 1, ticid  
      assert len(tc7) == 1, ticid
      assert tc['RA'].values == tc7['RA'].values
      assert tc['Dec'].values == tc7['Dec'].values
      assert tc['Tmag'].values == tc7['Tmag'].values
      assert tc['Epoc'].values == tc7['Epoc'].values
      assert tc['Period'].values == tc7['Period'].values
      assert tc['Duration'].values == tc7['Duration'].values
      assert tc['Transit_Depth'].values == tc7['Transit_Depth'].values
      assert (tc['star_rad'].isna().values == tc7['star_rad'].isna().values) or tc['star_rad'].values == tc7['star_rad'].values
      assert (tc['star_mass'].isna().values == tc7['star_mass'].isna().values) or tc['star_mass'].values == tc7['star_mass'].values
      assert (tc['teff'].isna().values == tc7['teff'].isna().values) or tc['teff'].values == tc7['teff'].values
      assert (tc['logg'].isna().values == tc7['logg'].isna().values) or tc['logg'].values == tc7['logg'].values
      assert tc['SN'].values == tc7['SN'].values
      assert tc['Qingress'].values == tc7['Qingress'].values
      assert (len(obj_type) == 0) or (obj_type.values[0] == 'STAR')
    else:
      if len(obj_type.values) and obj_type.values[0] == 'STAR':
        tc = origtce[origtce['tic_id'] == ticid]
        if not len(tc):
          tc = origtce2[origtce2['tic_id'] == ticid]
        assert len(tc), ticid
        assert len(tc['Ilabel'].values), ticid
        instar = tc['Ilabel'].values[0]
        if instar:
          split = 'instar'
        else:
          if rec['Final'] == 'U':
            split = 'instar'
          else:
            split = random.choice(['train', 'teat', 'val'])
      else:
        tc = origtce[origtce['tic_id'] == ticid]
        if not len(tc):
          tc = origtce2[origtce2['tic_id'] == ticid]
        split = 'notstar'
        assert len(tc), ticid
    assert 'filename' not in tc

  _, tc = next(tc.iterrows())
  
  rec['RA'] = tc['RA']
  rec['Dec'] = tc['Dec']
  rec['Tmag'] = tc['Tmag']
  rec['Epoc'] = tc['Epoc']
  rec['Per'] = tc['Period']
  rec['Dur'] = tc['Duration']
  rec['Depth'] = tc['Transit_Depth']
  rec['SRad'] = tc['star_rad']
  rec['SMass'] = tc['star_mass']
  
  rec['split'] = split
  
  if not isy3:
    if tc['disp_E'] > 0:
      assert rec['Final'] == 'E' or (
        ticid in (293346794,  # Changed later
                  294871044,  # Changed later
                  92328347,  # Changed later
                  83408987,  # Changed later
                  268608949,  # Changed later
                 )
        ), ticid
      assert tc['disp_S'] == 0
      assert tc['disp_B'] == 0
      assert tc['disp_J'] == 0
      assert tc['disp_N'] == 0
    elif tc['disp_S'] > 0:
      assert rec['Final'] == 'S'
      assert tc['disp_E'] == 0
      assert tc['disp_B'] == 0
      assert tc['disp_J'] == 0
      assert tc['disp_N'] == 0
    else:
      if tc['disp_B'] > 0:
        if rec['Final'] == 'B':
          assert tc['disp_E'] == 0
          assert tc['disp_S'] == 0
          assert tc['disp_J'] == 0
          assert tc['disp_N'] == 0
        else:
          assert np.isnan(rec['Final'])
          assert tc['disp_B'] == sum(rec[r] == 'B' for r in raters)
      if tc['disp_J'] > 0:
        if rec['Final'] == 'J':
          assert tc['disp_E'] == 0
          assert tc['disp_S'] == 0
          assert tc['disp_B'] == 0
          assert tc['disp_N'] == 0
        else:
          assert np.isnan(rec['Final'])
          assert tc['disp_J'] == sum(rec[r] == 'J' for r in raters), (idx, ticid)
      if tc['disp_N'] > 0:
        if rec['Final'] == 'N':
          assert tc['disp_E'] == 0
          assert tc['disp_S'] == 0
          assert tc['disp_B'] == 0
          assert tc['disp_J'] == 0
        else:
          assert np.isnan(rec['Final'])
          assert tc['disp_N'] == sum(rec[r] == 'N' for r in raters)
        
  prefs = {
    339672028: 's0013',
    425721385: 's0013',
    300557619: 's0012',
    327301957: 's0013',
    55559618:  's0012',
    351603103: 's0013',
    278866211: 's0012',
    391821647: 's0008',
    281979481: 's0013',
    314865962: 's0012',
    394722182: 's0013',
    383390264: 's0013',
    287156968: 's0013',
    167418903: 's0013',
    261257684: 's0013',
    311890977: 's0013',
    366989877: 's0013',
    38571020: 's0013',
    280097543: 's0013',
    373844472: 's0013',
    234345288: 's0013',
    261136679: 's0013',
    469782185: 's0013',
    300871545: 's0012',
    260304296: 's0012',
    401604346: 's0013',
    269450900: 's0013',
    295541511: 's0013',
    238197709: 's0012',
    421894914: 's0013',
    446549905: 's0013',
    6354567: '2019247',
    55452495: 's0013',
    263003176: 's0013',
    318608749: 's0013',
    253990973: 's0013',
    406976746: 's0013',
    278683844: 's0012',
    299799658: 's0013',
    271596418: 's0012',
    350153977: 's0013',
    143257766: 's0013',
    91576611: 's0013',
    300810086: 's0012',
    177077336: 's0013',
    278415929: 's0013',
    388130235: 's0013',
    425561347: 's0013',
    140760434: 's0013',
    382625239: 's0013',
    270341214: 's0013',
    41227743: 's0013',
    304100538: 's0013',
    179317684: 's0012',
    29831208: 's0012',
    348770361: 's0013',
    271900960: 's0012',
    370133522: 's0013',
    260985861: 's0013',
    375059587: 's0012',
    299780329: 's0013',
    380783252: 's0013',
    360742636: 's0013',
    287329267: 's0012',
    277099925: 's0012',
    325468685: 's0013',
    280095254: 's0013',
    382626661: 's0012',
    348844154: 's0012',
    339733013: 's0012',
    150428135: 's0013',
    261369656: 's0013',
    281924357: 's0012',
    409934330: 's0013',
    300293197: 's0013',
    372172128: 's0013',
    349373192: 's0012',
    271581073: 's0013',
    379286801: 's0013',
    306263608: '202107',
    155001079: 's0017',
    267489265: '2020262000000-s0016',
    116608612: '202107',
    417931607: 's0022',
    125735470: '202107',
    199376584: '2021070000000-s0015',
    188768068: 's0017',
    312083267: '2021070000000-s0015',
    387664866: '2021070000000-s0016',
    387834907: 's0025',
    277566483: '2021070000000-s0016',
    27491137: 's0023',
    375506058: '2021070000000-s0016',
    142387023: 's0022',
    115771549: '2021070000000-s0016',
    21755546: '2021070000000-s0014',
    69679391: '2021070000000-s0014',
    219857012: 's0026',
    136971594: '2021070000000-s0016',
    91987762: '2021070000000-s0021',
    353782445: 's0026',
    298663873: 's0026',
    172518755: '2021070000000-s0021',
    390651552: '2021070000000-s0023',
    352764091: 's0024',
    198384408: 's0026',
    1715469662: 's0026',
    138588540: 's0021',
    129979528: '2021070000000-s0018',
    311121985: '2021070000000-s0019',
    198213332: 's0026',
    207468071: 's0025',
    180695581: 's0023',
    356235833: '2021070000000-s0026',
    288132261: 's0026',
    287080092: 's0026',
    117789567: '2021070000000-s0014',
    350995812: '2021070000000-s0014',
    120602501: '2021070000000-s0016',
    356700488: 's0026',
    198187049: 's0026',
    284450803: 's0026',
    160268701: 's0021',
    142276270: 's0022',
    286923464: 's0022',
    21960306: 's0026',
    202563254: '2021070000000-s0015',
    118301361: '2021070000000-s0018',
    116483514: 's0024',
    298647682: 's0025',
    365683032: '2021070000000-s0016',
    365683032: '2021070000000-s0016',
    264171144: 's0026',
    368435330: '2021070000000-s0022',
    138819293: '2021070000000-s0022',
    233684293: '2021070000000-s0021',
    122220263: '2021070000000-s0014',
    1400770435: 's0026',
    202426247: '2021070000000-s0023',
    459978312: 's0026',
    269701147: 's0025',
    349827430: 's0021',
    192372961: '2021070000000-s0018',
    367366318: '2021070000000-s0019',
    26880783: '2021070000000-s0016',
    155114483: 's0017',
    275111900: '2021070000000-s0016',
    367607434: 's0017',
    137284119: '2021070000000-s0016',
    2676497: 's0019',
    395171208: '2021070000000-s0016',
    129539786: '2021070000000-s0016',
    377191482: '2021070000000-s0017',
    406672232: '2021070000000-s0014',
    354006740: '2021070000000-s0019',
    233009109: 's0026',
    237184773: 's0026',
    82308728: '2021070000000-s0022',
    198356533: 's0026',
    364186197: 's0025',
    69997672: '2021070000000-s0014',
    320525204: 's0026',
    298073824: '2021070000000-s0015',
    110428269: '2021070000000-s0014',
    89389197: '2021070000000-s0014',
    350132371: 's0025',
    219751469: 's0026',
    26547036: '2021070000000-s0016',
    28230919: '2021070000000-s0015',
    138017750: '2021070000000-s0018',
    293954617: '2021070000000-s0016',
    66445643: 's0017',
    224600500: 's0026',
    159510109: 's0026',
    16740101: '2021070000000-s0015',
    237222864: 's0026',
    198457103: 's0026',
    229455001: 's0026',
  }
#   minidx = 22470
  if isy3:
    rec['Year'] = 3
    rec['File'] = tc['filename']
    f = fits.open('/mnt/tess/lc/' + tc['filename'])
    rec['MinT'] = min(f[1].data["TIME"])
    rec['MaxT'] = max(f[1].data["TIME"])
  else:
    rec['Year'] = 2
    if (rec['Final'] == 'U'):
      rec['File'] = ''
      rec['MinT'] = -1
      rec['MaxT'] = -1
      rec['split'] = 'norpt'
    else:
      if ticid in prefs:
        pat = '/mnt/tess/lc/*' + prefs[ticid] + '*0' + str(ticid) + '*'
      else:
        pat = '/mnt/tess/lc/*0' + str(ticid) + '*'
      lc = glob.glob(pat)
      assert len(lc), pat
      flc = []
      for p in lc:
        t = time.localtime(os.path.getmtime(p))
        if (t.tm_year, t.tm_mon) < (2021, 8):
          flc.append(p)
      if len(flc) != 1:
        print(ticid, pat)
        print('\n'.join(flc))
        assert False
      flc, = flc
      rec['File'] = os.path.basename(flc)

      f = fits.open(flc)
      rec['MinT'] = min(f[1].data["TIME"])
      rec['MaxT'] = max(f[1].data["TIME"])
  
  final = final.append(rec)
  
  print(f'\r{idx}   ', end='')
#   if idx > 5:
#     break
  
final = final.astype({'Depth': int, 'TIC ID': int, 'Year': int})
  
final.sample()

final.to_csv('aligned.csv')

In [ ]:
pd.set_option('display.max_columns', None)
final